# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [431]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import requests
import json
import time
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [432]:
# City Data File (CSV)
data_file = "../WeatherPy/output_data/city_data.csv"
weather_data = pd.read_csv(data_file, dtype="object", encoding="utf-8")
weather_data.head(20)


,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Port Blair,IN,11.67,92.75,81.97,74,72,4.14,1595534773
1,Hobart,AU,-42.88,147.33,39.99,80,51,4.7,1595534753
2,Lebu,CL,-37.62,-73.65,48.31,62,46,5.3,1595534828
3,Hermanus,ZA,-34.42,19.23,57.99,85,0,3.0,1595534568
4,Atuona,PF,-9.8,-139.03,78.67,76,0,18.79,1595534829
5,Dikson,RU,73.51,80.55,40.98,87,0,7.07,1595534581
6,Barrow,US,71.29,-156.79,44.6,93,90,18.34,1595534550
7,Kaitangata,NZ,-46.28,169.85,37.99,79,98,5.01,1595534829
8,Vila Franca do Campo,PT,37.72,-25.43,69.8,68,40,5.82,1595534608
9,Qasigiannguit,GL,68.82,-51.19,53.6,62,75,3.36,1595534582


In [433]:
weather_data.count()

City               566
Country            565
Latitude           566
Longitude          566
Max Temperature    566
Humidity           566
Cloudiness         566
Wind Speed         566
Date               566
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [434]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [439]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)
# Convert Humidity to float and store
# HINT: be sure to handle NaN values
weather_data = weather_data.dropna()
humidity = weather_data['Humidity'].astype(float)


In [440]:
weather_data.count()

City               565
Country            565
Latitude           565
Longitude          565
Max Temperature    565
Humidity           565
Cloudiness         565
Wind Speed         565
Date               565
dtype: int64

In [441]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=np.max(humidity),
                                 point_radius = 3)
#add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [442]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph
#Zero cloudiness

weather_data['Max Temperature'] = weather_data['Max Temperature'].astype(float)
weather_data['Wind Speed'] = weather_data['Wind Speed'].astype(float)
weather_data['Cloudiness'] = weather_data['Cloudiness'].astype(int)
weather_data['Humidity'] = weather_data['Humidity'].astype(int)

narrowed_city = weather_data.loc[(weather_data['Max Temperature'] >= 70)  & (weather_data['Max Temperature'] < 85) & \
                                  (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0) & \
                                  (weather_data['Humidity'] <= 50)].dropna()

narrowed_city.head()

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
44,Kumluca,TR,36.37,30.29,84.52,43,0,4.76,1595534589
52,Curvelo,BR,-18.76,-44.43,78.58,31,0,7.34,1595534429
94,Cavalcante,BR,-13.8,-47.46,80.89,27,0,6.02,1595534600
121,San Jose,US,37.34,-121.89,82.00,44,0,3.06,1595534522
139,Porto Empedocle,IT,37.29,13.53,79.00,41,0,3.85,1595534611


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [443]:
hotel_df = narrowed_city[['City','Country','Latitude','Longitude']]
hotel_df['Hotel Name'] = ''
hotel_df

/Users/allyson/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Latitude,Longitude,Hotel Name
44,Kumluca,TR,36.37,30.29,
52,Curvelo,BR,-18.76,-44.43,
94,Cavalcante,BR,-13.8,-47.46,
121,San Jose,US,37.34,-121.89,
139,Porto Empedocle,IT,37.29,13.53,
244,Sabha,LY,27.04,14.43,
395,Kasongo-Lunda,CD,-6.47,16.82,
406,Tiznit Province,MA,29.58,-9.5,
441,Zhezkazgan,KZ,47.8,67.71,


In [444]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

response = requests.get(base_url,params=params).json()

In [445]:
for index, row in hotel_df.iterrows():
    # get city name and coordinate
    lat = row["Latitude"]
    lon = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lon}"

    # build url and make API request to Google Places 
    print(f"Retrieving Results for Index {index}: {city_name}.")
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_name = response.json()
    
    #print(json.dumps(hotel_results, indent=4, sort_keys=True))
    #Save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {hotel_name['results'][0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = hotel_name['results'][0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("----------------------------------------------\n")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)


Retrieving Results for Index 44: Kumluca.
Closest hotel in Kumluca is TurkuazKöy.
----------------------------------------------



/Users/allyson/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/allyson/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Retrieving Results for Index 52: Curvelo.
Closest hotel in Curvelo is Centerminas Palace Hotel.
----------------------------------------------

Retrieving Results for Index 94: Cavalcante.
Closest hotel in Cavalcante is Hostel Portal das Fadas.
----------------------------------------------

Retrieving Results for Index 121: San Jose.
Closest hotel in San Jose is Four Points by Sheraton San Jose Downtown.
----------------------------------------------

Retrieving Results for Index 139: Porto Empedocle.
Closest hotel in Porto Empedocle is Hotel Dioscuri Bay Palace.
----------------------------------------------

Retrieving Results for Index 244: Sabha.
Closest hotel in Sabha is فندق أفريقيا.
----------------------------------------------

Retrieving Results for Index 395: Kasongo-Lunda.
Missing field/result... skipping.
----------------------------------------------

Retrieving Results for Index 406: Tiznit Province.
Missing field/result... skipping.
------------------------------------

In [446]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
44,Kumluca,TR,36.37,30.29,TurkuazKöy
52,Curvelo,BR,-18.76,-44.43,Centerminas Palace Hotel
94,Cavalcante,BR,-13.8,-47.46,Hostel Portal das Fadas
121,San Jose,US,37.34,-121.89,Four Points by Sheraton San Jose Downtown
139,Porto Empedocle,IT,37.29,13.53,Hotel Dioscuri Bay Palace
244,Sabha,LY,27.04,14.43,فندق أفريقيا
395,Kasongo-Lunda,CD,-6.47,16.82,
406,Tiznit Province,MA,29.58,-9.5,
441,Zhezkazgan,KZ,47.8,67.71,Hotel Zumrat


In [447]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [451]:
#locations = locations.to_dict('split')

#locations = locations['data']
x = np.array(locations)
locations = x.astype(np.float)



In [452]:
# Add marker layer ontop of heat map
# Plot the hotels on top of the humidity heatmap with each pin containing 
# the Hotel Name, City, and Country.

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))